# Header Always Run For Every Notebook

In [1]:
import platform
print("python version " + platform.python_version())
import sys
import json
import os
import pandas as pd
from os.path import exists
from pathlib import Path
import logging

os.environ["CODE_BASE"] = "/Users/chenry/code"
sys.path = [os.environ.get("CODE_BASE","/scratch/shared/code")+"/chenry_utility_module/lib"] + sys.path
from chenry_utility_module.kbdevutils import KBDevUtils
kbdevutil = KBDevUtils("ModelSEED2")

from modelseedpy import MSPackageManager, MSModelUtil, MSBuilder, MSATPCorrection, MSGapfill,MSModelReport
from modelseedpy.core.mstemplate import MSTemplateBuilder
#You need to get a KBase dev token or pull a token from the narrative and put in ~/.kbase/token file

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
msrecon = kbdevutil.msseedrecon()

python version 3.9.13
KBBaseModules 0.0.1
Output files printed to:/Users/chenry/workspace/Notebooks//ModelSEED2/ when using KBDevUtils.out_dir()
modelseedpy 0.3.3
cobrakbase 0.3.1


# Predict central carbon minimal pathways for a set of genomes

In [3]:
model_workspace = 161565
#Get model list from model workspace
model_list = msrecon.kbase_api.list_objects(model_workspace, object_type="KBaseFBA.FBAModel", include_metadata=True)
output = {}
#Add code to read from the JSON file if it exists
if exists(KBDevUtils.out_dir()+"PathwayContent.json"):
    with open(KBDevUtils.out_dir()+"PathwayContent.json", 'r') as f:
        output = json.load(f)
for model_meta in model_list:
    if model_meta[1] not in output:
        output[model_meta[1]] = {}
        mdlutl = msrecon.get_model(str(model_workspace)+"/"+model_meta[1])
        atpcorrection = MSATPCorrection(mdlutl,msrecon.core_template,params["atp_medias"],load_default_medias=True,max_gapfilling=0,gapfilling_delta=0,forced_media=[],default_media_path=msrecon.module_dir+"/data/atp_medias.tsv")
        atpcorrection.evaluate_growth_media()
        media_list = atpcorrection.determine_growth_media()
        for media in media_list:
            mdlutl.model.objective = "rxn00062_c0"
            mdlutl.pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
            objective = mdlutl.model.slim_optimize()
            output[model_meta[1]][media.id] = {"ATP":objective,"reactions":{}}
            mdlutl.pkgmgr.getpkg("ObjConstPkg").build_package(objective, None)
            self.model.objective = self.model.problem.Objective(Zero, direction="min")
            obj_coef = dict()
            for reaction in self.model.reactions:
                if reaction.upper_bound > 0:
                    obj_coef[reaction.forward_variable] = 1
                if reaction.lower_bound < 0:
                    obj_coef[reaction.reverse_variable] = 1
            self.model.objective.set_linear_coefficients(obj_coef)
            solution = mdlutl.model.optimize()
            flux_values = mdlutl.compute_flux_values_from_variables()
            for rxnid in flux_values:
                if flux_values[reaction.id]["forward"] > 0.0000001:
                    output[model_meta[1]][media.id]["reactions"][rxnid] = ">"
                elif flux_values[reaction.id]["reverse"] > 0.0000001:
                    output[model_meta[1]][media.id]["reactions"][rxnid] = "<"
            mdlutl.pkgmgr.getpkg("ObjConstPkg").clear()
        with open(KBDevUtils.out_dir()+"PathwayContent.json", 'w') as f:
            f.write(output)

1701978529.189536 INFO: metabolites 1647
1701978529.5202281 INFO: reactions 1454
1701978530.296802 INFO: Default biomass: [bio1]


0.0


# Code for building an HTML report

In [2]:
mdlutl = msrecon.get_model("151253/GCF_000005845.2.RAST.Complete.mdl")
repobj = MSModelReport(mdlutl)
repobj.build_report(kbdevutil.out_dir()+"test.html")

1697049542.230461 INFO: metabolites 1586
1697049542.838496 INFO: reactions 1642
1697049543.3966238 INFO: The current solver interface glpk doesn't support setting the optimality tolerance.
1697049544.4482696 INFO: Default biomass: [bio1]
/scratch/shared/code/ModelSEEDpy/modelseedpy/core/msmodelreport.py:613: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(model_summary_df_styled.render(escape=False))
/scratch/shared/code/ModelSEEDpy/modelseedpy/core/msmodelreport.py:619: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(gapfillings_analysis_df_styled.render(escape=False))
/scratch/shared/code/ModelSEEDpy/modelseedpy/core/msmodelreport.py:630: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  f.write(atp_analysis_df_styled.render(escape=False))
